In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_log_error, mean_squared_error

In [2]:
seed=163

In [3]:
# Modelling ALgorithm

parametrs = {
    'max_leaf_nodes': [15, 20],
    'criterion': ['mse', 'mae']
}

FUNCTION = GridSearchCV(DecisionTreeRegressor(max_depth=5, 
                                              min_samples_split=2, 
                                              random_state=seed), 
                        parametrs)

FUNCTION_v2 = GridSearchCV(DecisionTreeRegressor(max_depth=5, 
                                              min_samples_split=2, 
                                              random_state=seed), 
                        parametrs)

# Data

In [4]:
train_df = pd.read_csv("../../Data/Fixed-Fixed.csv")
train_df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.0,0,0,1,52.976,18.084,9.0351
1,0.0,0,0,2,145.920,17.293,9.3771
2,0.0,0,0,3,285.990,17.430,9.5803
3,0.0,0,0,4,323.400,28.366,10.8720
4,0.0,0,0,5,472.810,17.537,9.7190


In [5]:
test_df = pd.read_csv("../../Data/Fixed-Fixed-Test.csv")
test_df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.5,63,510,1,52.613,7.5540,3.8030
1,0.5,63,510,2,144.790,7.2355,3.6191
2,0.5,63,510,3,283.540,7.3149,4.0421
3,0.5,63,510,4,322.530,11.8550,4.5453
4,0.5,63,510,5,468.260,7.4041,3.8655


In [6]:
train_obj = train_df.groupby("Mode")
test_obj = test_df.groupby("Mode")

# Utility

In [7]:
class Experiment:
    def __init__(self, data, x_args, y_args, split, seed):
        x = data[x_args].values
        y = data[y_args].values
        _X_train, _X_test, _y_train, self.Y_test = train_test_split(x, y, 
                                                                     test_size=split, 
                                                                     random_state=seed)
        
        self.x_args = x_args
        self.y_args = y_args
        
        self.std_X = StandardScaler()
        self.X_train = self.std_X.fit_transform(_X_train)
        self.X_test = self.std_X.transform(_X_test)
        
        self.std_Y = StandardScaler()
        self.Y_train = self.std_Y.fit_transform(_y_train)
        
        self.model = None
        
        
    def print_result(self, pred, true):
        for i in range(true.shape[-1]):
            print("- Explained Variance Score: {:0.6f}%".format(100*explained_variance_score(true[:,i], pred[:,i])))
            print("- Mean Absolute Error: {:0.6f}".format(mean_absolute_error(true[:,i], pred[:,i])))
            print("- Mean Squared Error: {:0.6f}".format(mean_squared_error(true[:,i], pred[:,i])))
            print("")
        
    def train_model(self, algorithm):
        self.model = algorithm
        self.model.fit(self.X_train, self.Y_train)
        
    def validation(self):
        res = self.model.predict(self.X_test)
        preds = self.std_Y.inverse_transform(res)
        preds = preds.reshape(self.Y_test.shape)
        
        print("\nVALIDATION RESULT")
        self.print_result(preds, self.Y_test)
        
    def testing(self, data):
        x = data[self.x_args].values
        y = data[self.y_args].values
        
        x = self.std_X.transform(x)
        
        res = self.model.predict(x)
        preds = self.std_Y.inverse_transform(res)
        preds = preds.reshape(y.shape)
        
        print("\nTEST RESULT")
        self.print_result(preds, y)

# Multi Modes Model

In [8]:
temp = Experiment(train_df, 
                  x_args=['depth', 'l1', 'l2', 'Mode'], 
                  y_args=['Frequency'], 
                  split=0.10, 
                  seed=seed)

temp.train_model(FUNCTION)
temp.validation()
temp.testing(test_df)


VALIDATION RESULT
- Explained Variance Score: 99.993384%
- Mean Absolute Error: 0.836446
- Mean Squared Error: 1.749041


TEST RESULT
- Explained Variance Score: 99.982955%
- Mean Absolute Error: 0.986388
- Mean Squared Error: 4.513858



# Single Mode Models

In [9]:
# Data Transformation

def transform(dataframe):
    dataframe_obj = dataframe.groupby("Mode")
    temp = deepcopy(dataframe_obj.get_group(1)[["depth", "l1", "l2", "Frequency"]])
    temp.pop("Frequency")
    temp = temp.reset_index()
    temp.pop("index")


    for i in range(1, 7):
        temp["Frequency_{}".format(i)] = 0
        temp["Frequency_{}".format(i)] = dataframe_obj.get_group(i)["Frequency"].reset_index()["Frequency"]
        
    return temp

In [10]:
train_df = transform(train_df)
test_df = transform(test_df)

In [11]:
train_df.head()

,depth,l1,l2,Frequency_1,Frequency_2,Frequency_3,Frequency_4,Frequency_5,Frequency_6
0,0.0,0,0,52.976,145.92,285.99,323.40,472.81,489.85
1,0.5,60,110,52.618,144.89,283.94,322.78,469.30,489.23
2,0.5,40,260,52.709,145.17,284.50,322.86,470.23,489.26
3,0.5,80,440,52.689,144.99,284.09,322.73,469.01,489.25
4,0.5,40,520,52.653,144.92,283.70,322.75,468.69,489.24


In [12]:
temp = Experiment(train_df, 
                  x_args=['depth', 'l1', 'l2'], 
                  y_args=['Frequency_1', 'Frequency_2', 'Frequency_3','Frequency_4', 'Frequency_5', 'Frequency_6'], 
                  split=0.10, 
                  seed=seed)

temp.train_model(FUNCTION_v2)
temp.validation()
temp.testing(test_df)


VALIDATION RESULT
- Explained Variance Score: 87.083997%
- Mean Absolute Error: 0.082159
- Mean Squared Error: 0.015290

- Explained Variance Score: 90.328308%
- Mean Absolute Error: 0.198467
- Mean Squared Error: 0.066610

- Explained Variance Score: 91.255976%
- Mean Absolute Error: 0.442016
- Mean Squared Error: 0.302005

- Explained Variance Score: 54.849822%
- Mean Absolute Error: 0.323360
- Mean Squared Error: 0.251394

- Explained Variance Score: 87.627130%
- Mean Absolute Error: 0.812792
- Mean Squared Error: 0.954785

- Explained Variance Score: 88.973603%
- Mean Absolute Error: 0.249531
- Mean Squared Error: 0.209388


TEST RESULT
- Explained Variance Score: 35.915304%
- Mean Absolute Error: 0.156473
- Mean Squared Error: 0.076081

- Explained Variance Score: 36.367515%
- Mean Absolute Error: 0.632033
- Mean Squared Error: 0.871664

- Explained Variance Score: 38.081959%
- Mean Absolute Error: 1.095324
- Mean Squared Error: 3.462311

- Explained Variance Score: 13.690979%
- 

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (th